In [658]:
from pyspark import SparkContext, SparkConf
import numpy as np

sc = SparkContext.getOrCreate()
tr = sc.textFile("datasets/small.txt")
ts = sc.textFile("datasets/small.txt")
k = sc.broadcast(5)

In [659]:
cart = tr.cartesian(ts)

In [660]:
def distances (dt):
    test = dt[0].split(",")
    idts = list(map(float, test[0:7]))
    clts = int(test[7])
    train = dt[1].split(",")
    idtr = list(map(float, train[0:7]))
    cltr = int(train[7])
    
    dist = sum((p-q)**2 for p, q in zip(idts, idtr)) ** .5
    
    if idts == idtr:
        dist = float("inf")
    
    return (dt[0],(dist,cltr,clts))
    

In [661]:
dist = cart.map(distances)
dist.saveAsTextFile('dist')

In [662]:
def create_klist(value):
    x = [(float('inf'),0)]*k.value
    x[0] = value
    return x

def merge_klist(x, value):
    for i in range(len(x)):
        if value[0]<x[i][0]:
            for j in range(len(x)-1,i,-1):
                x[j]=x[j-1]
            x[i]=value
            break
    return x

def merge_combiners(x,y):
    l = 0
    for i in range(len(x)):
        if y[l][0]<x[i][0]:
            for j in range(len(x)-1,i,-1):
                x[j]=x[j-1]
            x[i]=y[l]
            l+=1
    return x


In [663]:
def create_klist(value):
    x = [(float('inf'),0)]*k.value
    x[0] = value
    return x

def merge_klist(x, value):
    for i in range(len(x)):
        if value[0]<x[i][0]:
            x.insert(i,value)
            x.pop()
            break
    return x

def merge_combiners(x,y):
    l = 0
    for i in range(len(x)):
        if y[l][0]<x[i][0]:
            x.insert(i,y[l])
            x.pop()
            l+=1
    return x

In [664]:
k_vals = dist.combineByKey(create_klist, merge_klist, merge_combiners)
k_vals.saveAsTextFile('k_vals')

In [665]:
def guess_class(dt):
    rclass = dt[1][1][2]
    freq = 0
    predict = 0
    for i in range(len(dt[1])):
        tfreq = 1
        tpredict = dt[1][i][1]
        for j in range(i+1,len(dt[1])):
            if tpredict == dt[1][j][1]:
                tfreq +=1
        if tfreq > freq:
            predict = tpredict
            freq = tfreq
            
    
    return (rclass,predict)

In [666]:
guess_class = k_vals.map(guess_class)
guess_class.saveAsTextFile('guess_class')

In [667]:
def correct(dt):
    if dt[0]==dt[1]:
        return 1
    else:
        return 0

In [668]:
correct = guess_class.map(correct)
accuracy = correct.mean()


In [669]:
accum = sc.accumulator(0)
correct.foreach(lambda x: accum.add(x))
accum.value

274

In [670]:
accuracy

0.8154761904761905

In [18]:
sorted(sc.sequenceFile("out").collect())

[(1, 'a'), (2, 'aa'), (3, 'aaa')]

In [7]:
data = [1, 2, 3, 4, 5]
counter = 0
rdd = sc.parallelize(data)

# Wrong: Don't do this!!
def increment_counter(x):
    global counter
    counter += x
rdd.foreach(increment_counter)

print("Counter value: ", counter)

Counter value:  0


array([1, 3, 9])

In [75]:
l = [(float('inf'),0,0)]*10

In [79]:
for i in range(len(l)):
    if 4<l[i][0]:
        l[i]=(7,0,0)

print(l)

[(7, 0, 0), (7, 0, 0), (7, 0, 0), (7, 0, 0), (7, 0, 0), (7, 0, 0), (7, 0, 0), (7, 0, 0), (7, 0, 0), (7, 0, 0)]


In [72]:
l[0]= 2

In [262]:
from pyspark import SparkContext, SparkConf
import numpy as np

sc = SparkContext.getOrCreate()
tr = sc.textFile("datasets/small.txt")
ts = sc.textFile("datasets/small.txt")
k = sc.broadcast(5)

In [263]:
cart = tr.cartesian(ts)

def distances (dt):
    test = dt[0].split(",")
    idts = list(map(float, test[0:7]))
    clts = int(test[7])
    train = dt[1].split(",")
    idtr = list(map(float, train[0:7]))
    cltr = int(train[7])
    
    dist = sum((p-q)**2 for p, q in zip(idts, idtr)) ** .5
    
    if idts == idtr:
        dist = float("inf")
    
    return (dt[0],(dist,cltr,clts))
    
dist = cart.map(distances)
dist.saveAsTextFile('dist')

    

In [264]:
def create_klist(value):
    x = [(float('inf'),0)]*k.value
    x[0] = value
    return x

def merge_klist(x, value):
    for i in range(len(x)):
        if value[0]<x[i][0]:
            x[i]=value
            break
    return x

def merge_combiners(x,y):
    return x

In [265]:
k_vals = dist.combineByKey(create_klist, merge_klist, merge_combiners)
k_vals.saveAsTextFile('k_vals')

In [69]:
sumCount

PythonRDD[21] at RDD at PythonRDD.scala:53

In [523]:
t=
for a in range(5,1,-1):
    print (a)


5
4
3
2


In [589]:
x = [1,2,3,4,5,6,7,8,9,10]
i = 3
for j in range(len(x)-1,i,-1):
    x[j]=x[j-1]
x[i]=18

In [572]:

data = sc.parallelize( [(0, 2.), (0, 4.), (1, 0.), (1, 10.), (1, 20.)] )

sumCount = data.combineByKey(lambda value: (value, 1),
                             lambda x, value: (x[0] + value, x[1] + 1),
                             lambda x, y: (x[0] + y[0], x[1] + y[1]))

In [573]:
sumCount.saveAsTextFile('sumCount')

In [574]:
x = [(float('inf'),0)]*8

In [576]:
len(x[0])

2

In [590]:
x

[1, 2, 3, 18, 4, 5, 6, 7, 8, 9]

In [591]:
x.insert(3,186)

In [592]:
x

[1, 2, 3, 186, 18, 4, 5, 6, 7, 8, 9]

In [593]:
x.pop()

9

In [594]:
x

[1, 2, 3, 186, 18, 4, 5, 6, 7, 8]